In [1]:
!pip install transformers pythainlp
!pip install sentence_transformers
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path1 = '/content/drive/MyDrive/SeniorProject66/CutParagraphWithANN_Fixed/train'
path2 = '/content/drive/MyDrive/SeniorProject66/CutParagraphWithANN_Fixed/test'
path3 = '/content/drive/MyDrive/SeniorProject66/CutParagraphWithANN_Fixed/validation'
path_tags = '/content/drive/MyDrive/SeniorProject66/Tags_Embedding'

In [4]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import numpy as np

# LM models
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
word_embedding_model = models.Transformer(model_name, max_seq_length=32)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode='cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
paths = [path1, path2, path3]
input = []
train = []  # vector รวมทุกแถวของไฟล์ฃ
test = []
val = []
n = 1
POS = [("ADJ",1),("ADP",2),("ADV",3),("AUX",4),("CCONJ",5),("DET",6),("INTJ",7),("NOUN",8),("NUM",9),("PART",10),("PRON",11),("PROPN",12),("PUNCT",13),("SCONJ",14),("VERB",15)]
DEP=[("nsubj",1),("nsubj:outer",2),("nsubj:pass",3),("obj",4),("iobj",5),("csubj",6),("csubj:outer",7),("csubj:pass",8),("ccomp",9),("xcomp",10),("obl",11),("obl:agent",12),("obl:arg",13),("obl:lmod",14),("obl:tmod",15),("orphan",16),("vocative",17),("expl",18),("expl:impers",19),("expl:pass",20),("expl:pv",21),("discourse",22),("dislocated",23),("advcl",24),("advcl:relcl",25),("advmod",26),("advmod:emph",27),("advmod:lmod",28),("discourse",29),("aux",30),("aux:pass",31),("cop",32),("mark",33),("nmod",34),("nmod:poss",35),("nmod:tmod",36),("appos",37),("nummod",38),("nummod:gov",39),("acl",40),("acl:relcl",41),("amod",42),("det",43),("det:numgov",44),("det:nummod",45),("det:poss",46),("clf",47),("case",48),("conj",49),("cc",50),("cc:preconj",51),("fixed",52),("flat",53),("flat:foreign",54),("flat:name",55),("list",56),("parataxis",57),("compound",58),("compound:lvc",59),("compound:prt",60),("compound:redup",61),("compound:svc",62),("orphan",63),("goeswith",64),("reparandum",65),("punct",66),("ROOT",67),("dep",68)]
NER = [("O",0),("B-DATE",1),("I-DATE",1),("B-TIME",2),("I-TIME",2),("B-EMAIL",3),("I-EMAIL",3),("B-LEN",4),("I-LEN",4),("B-LENGTH",4),("I-LENGTH",4),("B-LOCATION",5),("I-LOCATION",5),("B-ORGANIZATION",6),("I-ORGANIZATION",6),("B-PERSON",7),("I-PERSON",7),("B-PHONE",8),("I-PHONE",8),("B-TEMPERATURE",9),("I-TEMPERATURE",9),("B-URL",10),("I-URL",10),("B-ZIP",11),("I-ZIP",11),("B-MONEY",12),("I-MONEY",12),("B-LAW",13),("I-LAW",13),("B-PERCENT",14),("I-PERCENT",14)]
ANN = [("O",0),("B-Person",1),("B-Place",2),("B-Property",3),("B-Evidence",4),("B-Frequency",5),("B-Company",6),("B-Damage",7),("B-Trading",8),("B-Vehicle",9),("B-Role",10),("B-Death",11),("B-Failure",12),("B-Allow",13),("B-Co-op",14),("B-Communicate",15),("B-Lend",16),("B-Help",17),("B-Have",18),("B-Intend",19),("B-Move",20),("B-Mistake",21),("B-Make",22),("B-Kill",23),("B-Crime",24),("B-Injure",25),("I-Person",26),("I-Place",27),("I-Property",28),("I-Evidence",29),("I-Frequency",30),("I-Company",31),("I-Damage",32),("I-Trading",33),("I-Vehicle",34),("I-Role",35),("I-Death",36),("I-Failure",37),("I-Allow",38),("I-Co-op",39),("I-Communicate",40),("I-Lend",41),("I-Help",42),("I-Have",43),("I-Intend",44),("I-Move",45),("I-Mistake",46),("I-Make",47),("I-Kill",48),("I-Crime",49),("I-Injure",50)]
POS_dict = dict(POS)
DEP_dict = dict(DEP)
NER_dict = dict(NER)
ANN_dict = dict(ANN)

for path in paths:
  for filename in os.listdir(path):

    if filename.endswith(".txt"):
      print(n,filename)
      file_path = os.path.join(path, filename)
      with open(file_path, 'r', encoding='utf-8') as file:
        deka = []
        vectors = []
        features = []
        output = []
        for line in file:           #เข้าทีละบรรทัด
          word_vectors = []         #เวคเตอร์รวม features (หนึ่งแถวของไฟล์)
          data = line.strip().split('\t')
          #print('data :',data)
          if data[0] == '':
            deka.append([vectors,features,output])
            output = []
            features = []
            vectors = []
            continue
          if len(data) >= 5:
            word = data[0]
            pos_tag = data[1]
            dep_tag = data[2]
            ner_tag = data[3]
            ann_tag = data[4]
          else:
            continue

          pos_value = POS_dict.get(pos_tag,0)
          dep_value = DEP_dict.get(dep_tag,0)
          ner_value = NER_dict.get(ner_tag,0)
          ann_value = ANN_dict.get(ann_tag,0)
          #print((pos_value), dep_value, ner_value, ann_value)

          word_vector = model.encode(word,normalize_embeddings=True) #embedding คำ
          features.append(np.array([np.array(pos_value),np.array(dep_value),np.array(ner_value)]))
          word = np.array(word_vector)
          output.append(np.array(ann_value))

          data[1] = pos_value  # แทนคำที่สองด้วยตัวเลขที่มีคู่ค่าของคำนั้น
          data[2] = dep_value
          data[3] = ner_value
          data[4] = ann_value
          vectors.append(word)
          #print("convert :",data,'\n')
      input.append(deka)
      #break
    n+=1
    #if(n==5):
    #  break
  if(path == paths[0]):
    train = input
  elif path == paths[1]:
    test = input
  elif path == paths[2]:
    val = input
  input = []

1 2097_2530.txt
2 2442_2527.txt
3 2240_2533.txt
4 1265_2529.txt
5 1172_2505.txt
6 1136_2529.txt
7 1006_2542.txt
8 2105_2522.txt
9 1155_2520.txt
10 101_2524.txt
11 1048_2512.txt
12 1119_2502.txt
13 1225_2515.txt
14 1283_2514.txt
15 2483_2528.txt
16 2216_2530.txt
17 1186_2542.txt
18 2307_2530.txt
19 2431_2544.txt
20 1161_2549.txt
21 2397_2527.txt
22 2403_2531.txt
23 1219_2505.txt
24 2297_2514.txt
25 1093_2517.txt
26 1238_2515.txt
27 1240_2504.txt
28 118_2515.txt
29 2138_2521.txt
30 2377_2515.txt
31 101_2533.txt
32 1281_2493.txt
33 1306_2513.txt
34 1269_2513.txt
35 1088_2540.txt
36 1185_2529.txt
37 2418_2530.txt
38 1141_2514.txt
39 1101_2509.txt
40 2396_2527.txt
41 2301_2528.txt
42 2528_2515.txt
43 1158_2529.txt
44 117_2515.txt
45 1057_2511.txt
46 2513_2520.txt
47 2473_2520.txt
48 1246_2526.txt
49 1065_2514.txt
50 1024_2501.txt
51 1281_2508.txt
52 2283_2515.txt
53 1273_2515.txt
54 1088_2519.txt
55 1113_2542.txt
56 2223_2515.txt
57 1014_2535.txt
58 1067_2502.txt
59 1024_2519.txt
60 2257_25

In [ ]:
print("total train file :",len(train))
print("total test file :",len(test))
print("total validation file :",len(val))

total_sen_train = sum(len(i) for i in train)
total_sen_test = sum(len(i) for i in test)
total_sen_val = sum(len(i) for i in val)
print("total sentences of train/test/validation :",total_sen_train,total_sen_test,total_sen_val)

print("Example X1 Input :\n",(((train[0][0][0][0]))))   #ไฟล์แรก,ประโยคแรก,ดู input แรก (เวกเตอร์),ดูคำแรก
print("Example X2 Input :\n",(((train[0])[0][1][0])))   #ไฟล์แรก,ประโยคแรก,ดู input ที่สอง (ฟีเจอร์),ดูฟีเจอร์ของคำแรก (pos, dep, ner)
print("Example Y Input :\n",(((train[0])[0][2][0])))   #ไฟล์แรก,ประโยคแรก,ดู output (ann),ดู tag ของคำแรก

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 195

X1_train = pad_sequences([np.array(item[0]) for sublist in train for item in sublist], maxlen=max_length, dtype='float32', padding='post')
X2_train = pad_sequences([np.array(item[1]) for sublist in train for item in sublist], maxlen=max_length, dtype='float32',padding='post')

X1_test = pad_sequences([np.array(item[0]) for sublist in test for item in sublist], maxlen=max_length, dtype='float32',padding='post')
X2_test = pad_sequences([np.array(item[1]) for sublist in test for item in sublist], maxlen=max_length, dtype='float32',padding='post')

y_train = pad_sequences([np.array(item[2]) for sublist in train for item in sublist], maxlen=max_length, dtype='float32',padding='post')
y_test = pad_sequences([np.array(item[2]) for sublist in test for item in sublist], maxlen=max_length, dtype='float32',padding='post')

X1_val_train = pad_sequences([np.array(item[0]) for sublist in val for item in sublist], maxlen=max_length, dtype='float32',padding='post')
X2_val_train = pad_sequences([np.array(item[1]) for sublist in val for item in sublist], maxlen=max_length, dtype='float32',padding='post')
y_val = pad_sequences([np.array(item[2]) for sublist in val for item in sublist], maxlen=max_length, dtype='float32',padding='post')

#y_train = to_categorical(y_train, num_classes=len(ANN))
#y_test = to_categorical(y_test, num_classes=len(ANN))
#y_val = to_categorical(y_val, num_classes=len(ANN))
y_train = pad_sequences([np.array(item[2]) for sublist in train for item in sublist], maxlen=max_length, dtype='float32',padding='post')
y_test = pad_sequences([np.array(item[2]) for sublist in test for item in sublist], maxlen=max_length, dtype='float32',padding='post')
y_val = pad_sequences([np.array(item[2]) for sublist in val for item in sublist], maxlen=max_length, dtype='float32',padding='post')
#print((t[0][194]))
#print((y_train[0]))
#print(y_test[0])
print('len ANN:',len(ANN))
print((X1_train[0][0]))
print((X2_train[0][0]))
print((y_train[0][0]))

In [ ]:
from keras.models import load_model

# โหลดโมเดลจากไฟล์
loaded_model = load_model('/content/drive/MyDrive/SeniorProject66/Bi-LSTM_1.h5')

# ทำนายข้อมูลด้วยโมเดลที่โหลด
predictions = loaded_model.predict([X1_test, X2_test])
loaded_model.summary()
loss, accuracy = loaded_model.evaluate([X1_test, X2_test], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

true_classes = np.argmax(y_test, axis=2)
confusion_mtx = confusion_matrix(true_classes, np.argmax(predictions, axis=2))
print(confusion_mtx)
import matplotlib.pyplot as plt
import seaborn as sns

# กำหนดชื่อคลาส
class_names = [label[0] for label in ANN]

# พล็อตเมตริกการสับเปลี่ยนในรูปแบบของ Heatmap
plt.figure(figsize=(30, 30))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Classes')
plt.ylabel('True Classes')
plt.title('Confusion Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=45)

save_pic_path = '/content/drive/MyDrive/SeniorProject66/Result3.png'
plt.savefig(save_pic_path)
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Bidirectional, TimeDistributed, Embedding, InputLayer, Concatenate, Input
from keras.preprocessing.sequence import pad_sequences

#print(X2_train[0])
input_layer1 = Input(shape=(max_length, 768))
input_layer2 = Input(shape=(max_length, 3))

x1 = Bidirectional(LSTM(768, return_sequences=True))(input_layer1)

x2 = Bidirectional(LSTM(500, return_sequences=True))(x1)

x3 = Bidirectional(LSTM(250, return_sequences=True))(x2)

x4 = Bidirectional(LSTM(125, return_sequences=True))(x2)

x5 = Bidirectional(LSTM(60, return_sequences=True))(x2)

x6 = Bidirectional(LSTM(20, return_sequences=True))(x4)

x7 = Bidirectional(LSTM(3, return_sequences=True))(input_layer2)

concatenated = Concatenate()([x1, x2, x3, x4, x5, x6, x7])

output = (Dense(1, activation='sigmoid'))(concatenated)

model = Model(inputs=[input_layer1, input_layer2], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()

model.fit([X1_train, X2_train], y_train, batch_size=32, epochs=10, validation_data=([X1_val_train, X2_val_train], y_val))

loss, accuracy = model.evaluate([X1_test, X2_test], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


In [ ]:
import numpy as np

# หาคลาสที่ถูกทำนายมากที่สุดสำหรับแต่ละตัวอย่าง
predicted_classes = np.argmax(y_train, axis=1)

# นับจำนวนข้อมูลในแต่ละคลาส
class_labels, class_counts = np.unique(y_test, return_counts=True)
for label, count in zip(class_labels, class_counts):
    print(f"Class {label}: {count} samples")

# หาค่าน้ำหนักสำหรับแต่ละคลาส
total_samples = len(predicted_classes)
class_weights = total_samples / (len(class_labels) * class_counts)
print("Class weights:")
print(class_weights)

import numpy as np

# พิมพ์ค่าทั้งหมดใน y_train
y_train_restored = np.argmax(y_train, axis=1)
print("All values in y_train:", np.unique(y_train_restored))


In [ ]:
#save_pic_path = '/content/drive/MyDrive/SeniorProject66/Result1.png'
#plt.savefig(save_pic_path)

model.save('/content/drive/MyDrive/SeniorProject66/Bi-LSTM_2.h5')
